<a href="https://colab.research.google.com/github/tombraganca/distributed-computing/blob/main/Tarefa9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install asyncio

In [ ]:
import asyncio

class LamportMutex:
    def __init__(self, num_processes):
        self._num_processes = num_processes
        self._lamport_clocks = [0] * num_processes
        self._is_waiting = [False] * num_processes
        self._completers = [asyncio.Future() for _ in range(num_processes)]

    async def lock(self, process_id):
        self._is_waiting[process_id] = True
        self._lamport_clocks[process_id] = self._get_max_clock_value() + 1
        self._is_waiting[process_id] = False

        for i in range(self._num_processes):
            if i != process_id:
                await self._send_request(process_id, i)
                await self._receive_reply(i, process_id)

    def unlock(self, process_id):
        self._lamport_clocks[process_id] += 1
        if not self._completers[process_id].done():
            self._completers[process_id].set_result(None)

    async def _send_request(self, sender, receiver):
        await asyncio.sleep(0.1)  # Simulando o envio de uma solicitação
        print(f'Processo {sender} enviou solicitação para processo {receiver}')

    async def _receive_reply(self, receiver, sender):
        await asyncio.sleep(0.1)  # Simulando a recepção de uma resposta
        print(f'Processo {receiver} recebeu solicitação de processo {sender}')
        if not self._completers[receiver].done():
            self._completers[receiver].set_result(None)

    def _get_max_clock_value(self):
        return max(self._lamport_clocks)

async def main():
    num_processes = 3
    mutex = LamportMutex(num_processes)

    for i in range(num_processes):
        await mutex.lock(i)
        print(f'Processo {i} está na seção crítica')
        mutex.unlock(i)

# Executando o exemplo
await main()

Processo 0 enviou solicitação para processo 1
Processo 1 recebeu solicitação de processo 0
Processo 0 enviou solicitação para processo 2
Processo 2 recebeu solicitação de processo 0
Processo 0 está na seção crítica
Processo 1 enviou solicitação para processo 0
Processo 0 recebeu solicitação de processo 1
Processo 1 enviou solicitação para processo 2
Processo 2 recebeu solicitação de processo 1
Processo 1 está na seção crítica
Processo 2 enviou solicitação para processo 0
Processo 0 recebeu solicitação de processo 2
Processo 2 enviou solicitação para processo 1
Processo 1 recebeu solicitação de processo 2
Processo 2 está na seção crítica
